# Robustness metric and classifier

FI = feature importance
FS = Feature selection
FG = faking good
FB = faking bad

## Prof questions and thoughts

1) As regards to the importance of the features (items of the tests). After feature selection, Do diverse ML models with equal accuracy index different items as more important? (in discriminating Honest vs dishonest responses)

	ATTENTION: take note that in some datasets the Dishonest is fake-good ad in others i fake-bad.
Is there a difference in response to question n. 1 in relation to fake-good/Fake bad?


2) Does the response to question 1  change according to the dataset?


3) Which is the accuracy  in classification that is achieved using only the best 20% of the items?
Which is the amount of shrinkage with respect to the original test?  (e.g. with the full test of 100 Random forest lands an accuracy of 92% with the best 20 items the accuracy drops to 88%)
 If you take the best 20% of the items of a given test,  which is the concordance between classifiers in spotting the 20% best items? (e.g. in a test of 100 items only 10 among the best 20 are the same for different classifiers ; SVM Random Forest, knn etc.). You may use the rank order correlation


4) Are model agnostic feature selection (e.g. permutation importance or PCA) procedures better than model dependent


5) Are psychometric inspired items selection better than model based item selection?
Psychometric inspired item selection are usually based on PCA or factor analysis. With the best items selected on the basis of factor loadings.


6) Comments

## Dataset
- Short DT: 2 dataset faking good (27 features x ~400 osservazioni)
- PRMQ: 1 dataset faking bad (16 feature x 1400 osservazioni)
- PCL: 1 dataset faking bad (20 feature x 400 osservazioni)
- NAQ: 1 dataset faking bad (21 feature x 800 osservazioni)
- GAD: 1 dataset faking bad (16 feature x 1100 osservazioni)
- PID: 1 dataset faking bad (220 feature x 400 osservazioni)
- Short PID: 1 dataset faking bad (25 feature x 1100 osservazioni)
- PRFQ: 1 dataset faking good (18 feature x 678 osservazioni)
- IESR: 1 dataset faking bad (22 feature x 378 osservazioni)
- NEO PI: 1 dataset faking good (30 feature x 80000 osservazioni)
- DDDT: 1 dataset faking good (15 feature x 1000 osservazioni)
- IADQ: 1 dataset faking bad (9 feature x 450 osservazioni)
- BF: 3 dataset faking good (10 feature x ~230 osservazioni)

## Import data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from tqdm import trange

In [ ]:
N_CV = 5
REMOVE_COL = True
PERCENTAGE_FEATURE_TO_REMOVE = .8
COV_THRESHOLD = .5

In [ ]:
dataset = pd.read_csv("1shortDT//DT_df_CC.csv")
#dataset = pd.read_csv("1shortDT//DT_df_JI.csv")
#dataset = pd.read_csv("2PRMQ//PRMQ_df.csv") # senza correlazioni toglie tutte le domande meno che due...
#dataset = pd.read_csv("3PCL//PCL5_df.csv") # senza correlazioni toglie tutte le domande meno che due...
#dataset = pd.read_csv("6PID5//PID5_df.csv") # tante feature, si spacca parecchia roba
#dataset = pd.read_csv("7shortPID5//sPID-5_df.csv") # dimostra che KPCA honest only non è una buona scelta
#dataset = pd.read_csv("8PRFQ//PRFQ_df.csv")
#dataset = pd.read_csv("9IESR//IESR_df.csv") # ha molta covarianza, probabilmente la più alta

In [ ]:
data = np.array(dataset)

Assumption: the variables are monotonic wrt the response, so no "to categorical" is needed

In [ ]:
X = data[:,:-1].astype(float) # StandardScaler().fit_transform(data[:,:-1].astype(float))
y = np.zeros(data.shape[0])
y[data[:,-1] == "H"] = 1
original_dataset_size = X.shape[-1]

## Exploratory analysis

The more separated they are, the simpler is the task, as for example a Naybe bayesian yould capture the difference easily since assumes independence in the variables

### Features marginal distributions

In [ ]:
import seaborn as sns

fig, axs = plt.subplots(9,3, figsize=(15,15))
fig.tight_layout(pad=1.5)
for i, (ax, label_name) in enumerate(zip(axs.flatten(), dataset.columns[:-1])):
	sns.kdeplot(X[np.where(y == 1)][:,i], ax=ax)
	sns.kdeplot(X[np.where(y == 0)][:,i], ax=ax)
	ax.set_xlim(1,5)
	ax.set_xticks(range(1,6))
	ax.set_title(label_name)

### Features mean and std

In [ ]:
m1 = X[y==1].mean(axis=0)
m0 = X[y==0].mean(axis=0)
e1 = X[y==1].std(axis=0)
e0 = X[y==0].std(axis=0)
plt.fill_between(range(1, X.shape[-1]+1), m1-e1, m1+e1, alpha=0.5, label="_nolegend_")
plt.plot(range(1, X.shape[-1]+1), m1)

plt.fill_between(range(1, X.shape[-1]+1), m0-e0, m0+e0, alpha=0.5, label="_nolegend_")
plt.plot(range(1, X.shape[-1]+1), m0)

plt.ylabel("Mean Likert value")
plt.xlabel("Feature (Question number)")
plt.legend(["Honest", "Dishonest"])

plt.show()

## Uncorrelation

In [ ]:
from scipy import stats

In [ ]:
if REMOVE_COL:
	while X.shape[-1] > 1:
		corr = stats.spearmanr(X).correlation
		if isinstance(corr, np.float64):
			corr = np.array([corr])
		correlations = np.where(np.sum(
			np.abs(corr*(1-np.identity(len(corr)))) > COV_THRESHOLD
			, axis=1) > 0)[0]
		if len(correlations) > 0:
			X = X[:,np.where(np.arange(0, X.shape[-1]) != correlations[0])[0]]
			print(f"deleting {correlations[0]}")
		else:
			break

In [ ]:
X.shape[-1],int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))

In [ ]:
if X.shape[-1] < int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE)):
	raise Exception("not enough feature, increase covariance threshold or decrease % features to remove")

## FS with model dependent

In [ ]:
# with model-dependent feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

In [ ]:
# take feature importance and transform them to distribution (positive and sum to 1)
def to_distribution(values):
	values = np.abs(values)
	return values / np.sum(values) if np.sum(values) > 0 else np.array([1])

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(X,y)
logistic_regression = LogisticRegression(random_state=0).fit(X,y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(X,y)
svm = SVC(kernel = "linear",random_state=0).fit(X,y)

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .2
plt.figure(dpi=200, figsize=(20,5))
plt.bar(x_bar, to_distribution(random_forest.feature_importances_), width=width, color='yellow')
plt.bar(x_bar+width, to_distribution(logistic_regression.coef_.squeeze()), width=width, color='greenyellow')
plt.bar(x_bar+width*2, to_distribution(gradient_boosting.feature_importances_), width=width, color='yellowgreen')
plt.bar(x_bar+width*3, to_distribution(svm.coef_.squeeze()), width=width, color='olive')
plt.ylabel("Feature importance (%)")
plt.xlabel("Question number")
plt.legend(["Random Forest", "Logistic Regression", "Gradient Boosting", "SVM"])
_ = plt.xticks(x_bar + width + width / 2, x_bar)

Accuracy using all the features:

In [ ]:
print(f"""
Accuracies:
Random Forest: {(cross_val_score(random_forest, X, y, cv=N_CV).mean()*100).round(2)}%
Logistic Regression: {(cross_val_score(logistic_regression, X, y, cv=N_CV).mean()*100).round(2)}%
Gradient Boosting: {(cross_val_score(gradient_boosting, X, y, cv=N_CV).mean()*100).round(2)}%
SVC: {(cross_val_score(svm, X, y, cv=N_CV).mean()*100).round(2)}%
""")

Accuracy using only top 20% specific for each model

In [ ]:
random_forest_top_20_features = np.argsort(to_distribution(random_forest.feature_importances_))[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
logistic_regression_top_20_features = np.argsort(to_distribution(logistic_regression.coef_.squeeze()))[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
gradient_boosting_top_20_features = np.argsort(to_distribution(gradient_boosting.feature_importances_))[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
svm_top_20_features = np.argsort(to_distribution(svm.coef_.squeeze()))[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]

print(f"""
Accuracies:
Random Forest: {(cross_val_score(random_forest, X[:,random_forest_top_20_features], y, cv=N_CV).mean()*100).round(2)}%
Logistic Regression: {(cross_val_score(logistic_regression, X[:,logistic_regression_top_20_features], y, cv=N_CV).mean()*100).round(2)}%
Gradient Boosting: {(cross_val_score(gradient_boosting, X[:,gradient_boosting_top_20_features], y, cv=N_CV).mean()*100).round(2)}%
SVC: {(cross_val_score(svm, X[:,svm_top_20_features], y, cv=N_CV).mean()*100).round(2)}%
""")

taking top 20%, which is the amount of concordance between them:

In [ ]:
np.array((random_forest_top_20_features, logistic_regression_top_20_features, gradient_boosting_top_20_features, svm_top_20_features))

Change in accuracy considering only top N features ordered by importance wrt each model

In [ ]:
random_forest_sorted_features = np.argsort(to_distribution(random_forest.feature_importances_)).squeeze()
logistic_regression_sorted_features = np.argsort(to_distribution(logistic_regression.coef_.squeeze())).squeeze()
gradient_boosting_sorted_features = np.argsort(to_distribution(gradient_boosting.feature_importances_)).squeeze()
svm_sorted_features = np.argsort(to_distribution(svm.coef_.squeeze())).squeeze()
accuracies_filtered_singularly = []
X_filtered_rf = np.copy(X)
X_filtered_lg = np.copy(X)
X_filtered_svm = np.copy(X)
X_filtered_gb = np.copy(X)
for i in trange(0, X.shape[-1]):

	# find accuracy
	accuracies_filtered_singularly.append([
		(cross_val_score(random_forest, X_filtered_rf, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered_lg, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered_gb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered_svm, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])

	# train to get feature importance
	random_forest.fit(X_filtered_rf, y)
	logistic_regression.fit(X_filtered_lg, y)
	gradient_boosting.fit(X_filtered_gb, y)
	svm.fit(X_filtered_svm, y)

	# from feature importance to distribution
	random_forest_feature_importance = to_distribution(random_forest.feature_importances_)
	logistic_regression_feature_importance = to_distribution(logistic_regression.coef_.squeeze())
	gradient_boosting_feature_importance = to_distribution(gradient_boosting.feature_importances_)
	svm_feature_importance = to_distribution(svm.coef_.squeeze())

 	# remove the least important feature for each model
	X_filtered_rf = X_filtered_rf[:, np.argsort(random_forest_feature_importance)[::-1][:-1]]
	X_filtered_lg = X_filtered_lg[:, np.argsort(logistic_regression_feature_importance)[::-1][:-1]]
	X_filtered_gb = X_filtered_gb[:, np.argsort(gradient_boosting_feature_importance)[::-1][:-1]]
	X_filtered_svm = X_filtered_svm[:, np.argsort(svm_feature_importance)[::-1][:-1]]


In [ ]:
plt.plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[::-1])
plt.xticks(np.arange(1, X.shape[-1]+1, 3))
plt.legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
plt.xlabel("Number of top feature considered in the training")
plt.ylabel("Accuracy (%)")
plt.show()

### Feature elimination based on aggregate importance

In [ ]:
accuracies_filtered_together = []
X_filtered = np.copy(X)
feature_importance = []
features_selected_at_each_step = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step = []
for i in trange(X.shape[-1], 0, -1):
	accuracies_filtered_together.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	if i > 1 :
		random_forest.fit(X_filtered, y)
		logistic_regression.fit(X_filtered, y)
		gradient_boosting.fit(X_filtered, y)
		svm.fit(X_filtered, y)

		random_forest_feature_importance = to_distribution(random_forest.feature_importances_)
		logistic_regression_feature_importance = to_distribution(logistic_regression.coef_.squeeze())
		gradient_boosting_feature_importance = to_distribution(gradient_boosting.feature_importances_)
		svm_feature_importance = to_distribution(svm.coef_.squeeze())

		summed_importance = np.sum([
			random_forest_feature_importance, logistic_regression_feature_importance,
			gradient_boosting_feature_importance, svm_feature_importance
		], axis=0)

		feature_importance.append(summed_importance)
		selected_features = np.argsort(summed_importance)[::-1][:-1]
		features_selected_at_each_step.append(selected_features)
		features_importance_at_each_step.append([
			random_forest_feature_importance, logistic_regression_feature_importance,
			gradient_boosting_feature_importance, svm_feature_importance
		])

		X_filtered = X_filtered[:,selected_features]

In [ ]:
_, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].set_title("Common feature selection")
axs[0].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together[::-1])
axs[0].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[0].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[0].set_xlabel("Number of top feature considered in the training")
axs[0].set_ylabel("Accuracy")

axs[1].set_title("Individual feature selection")
axs[1].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[::-1])
axs[1].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[1].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[1].set_xlabel("Number of top feature considered in the training")
axs[1].set_ylabel("Accuracy")

plt.show()

$$
Index = \frac{1}{n \choose 2} \sum_{(r_0, r_1) \subset R} Index(r_0, r_1)
$$


### Using an Voting ensamble

In [ ]:
voting_ensemble = VotingClassifier([
	('rf', random_forest),
	('lr', logistic_regression),
	('gb', gradient_boosting)
])

In [ ]:
accuracies_voting_ensemble = []
for features in features_selected_at_each_step:
	accuracies_voting_ensemble.append((cross_val_score(voting_ensemble, X[:,features], y, cv=N_CV, n_jobs=-1).mean()*100).round(2))

In [ ]:
_, axs = plt.subplots(1, 3, figsize=(30,5))
axs[0].set_title("Common feature selection")
axs[0].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together[::-1])
axs[0].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[0].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[0].set_xlabel("Number of top feature considered in the training")
axs[0].set_ylabel("Accuracy")

axs[1].set_title("Individual feature selection")
axs[1].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[::-1])
axs[1].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[1].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[1].set_xlabel("Number of top feature considered in the training")
axs[1].set_ylabel("Accuracy")

axs[2].set_title("Voting Ensemble Accuracy")
axs[2].plot(np.arange(1, X.shape[-1]+1), accuracies_voting_ensemble[::-1])
axs[2].plot(np.arange(1, X.shape[-1]+1), np.mean(accuracies_filtered_together[::-1], axis=-1))
axs[2].legend(["Voting ensemble accuracy","Average single model accuracy"])
axs[2].set_xlabel("Number of top feature considered in the training")
axs[2].set_ylabel("Accuracy")
_ = axs[2].set_xticks(np.arange(1, X.shape[-1]+1, 3))

### Using Stacking Ensemble (meta-learning)

In [ ]:
stacking_ensemble = StackingClassifier(estimators=[
	('rf', RandomForestClassifier(n_estimators=100)),
	('gb', gradient_boosting),
], final_estimator=logistic_regression)

In [ ]:
accuracies_stacking_ensemble = []
for features in features_selected_at_each_step:
	accuracies_stacking_ensemble.append((cross_val_score(stacking_ensemble, X[:,features], y, cv=N_CV, n_jobs=-1).mean()*100).round(2))

In [ ]:
_, axs = plt.subplots(1, 3, figsize=(30,5))
axs[0].set_title("Common feature selection")
axs[0].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together[::-1])
axs[0].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[0].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[0].set_xlabel("Number of top feature considered in the training")
axs[0].set_ylabel("Accuracy")

axs[1].set_title("Individual feature selection")
axs[1].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[::-1])
axs[1].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[1].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[1].set_xlabel("Number of top feature considered in the training")
axs[1].set_ylabel("Accuracy")

axs[2].set_title("Ensemble Accuracy")
axs[2].plot(np.arange(1, X.shape[-1]+1), np.array(accuracies_filtered_together[::-1])[:,0])
axs[2].plot(np.arange(1, X.shape[-1]+1), accuracies_voting_ensemble[::-1])
axs[2].plot(np.arange(1, X.shape[-1]+1), accuracies_stacking_ensemble[::-1])
axs[2].plot(np.arange(1, X.shape[-1]+1), np.mean(accuracies_filtered_together[::-1], axis=-1))
axs[2].legend(["Random Forest", "Voting ensemble accuracy","Stacking ensemble accuracy","Average single model accuracy"])
axs[2].set_xlabel("Number of top feature considered in the training")
axs[2].set_ylabel("Accuracy")
_ = axs[2].set_xticks(np.arange(1, X.shape[-1]+1, 3))

## FS with model independent with classifiers

here:
  	- permutation importance
  	- partial dependence
  	- leave one covariate out
  	- Information Gain

here and in the unsupervised section we can use more models as the techniques used to estimate the importance are model agnostic, so we don't need a clear estimator inside the model (reason why I just used those 4 models)

In [ ]:
# Extra, per iniziare confronteremo solo quelli con la baseline di sklearn (quelli fatti nella sezione prima)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier # categorical_features

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.inspection import partial_dependence

### Permutation importance

#### PI with singular classifiers

In [ ]:
# Features importances plot for each classifier

random_forest = RandomForestClassifier(random_state=0).fit(X,y)
logistic_regression = LogisticRegression(random_state=0).fit(X,y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(X,y)
svm = SVC(kernel = "linear",random_state=0).fit(X,y)

PI_random_forest = permutation_importance(random_forest, X, y, n_repeats=10, random_state=0)
PI_logistic_regression = permutation_importance(logistic_regression, X, y, n_repeats=10, random_state=0)
PI_gradient_boosting = permutation_importance(gradient_boosting, X, y, n_repeats=10, random_state=0)
PI_svm = permutation_importance(svm, X, y, n_repeats=10, random_state=0)

PI_random_forest_importance = to_distribution(PI_random_forest.importances_mean)
PI_logistic_regression_importance = to_distribution(PI_logistic_regression.importances_mean)
PI_gradient_boosting_importance = to_distribution(PI_gradient_boosting.importances_mean)
PI_svm_importance = to_distribution(PI_svm.importances_mean)

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .2
_, axs = plt.subplots(2, 2, figsize=(20,10))
axs[0][0].set_title("Random Forest feature selection")
axs[0][0].bar(x_bar, PI_random_forest_importance, width=width)
axs[0][0].bar(x_bar+width, to_distribution(random_forest.feature_importances_), width=width)
axs[0][0].legend(["PI importances", "Model importances"])
axs[0][1].set_title("Logistic regression feature selection")
axs[0][1].bar(x_bar, PI_logistic_regression_importance, width=width)
axs[0][1].bar(x_bar+width, to_distribution(logistic_regression.coef_.squeeze()), width=width)
axs[0][1].legend(["PI importances", "Model importances"])
axs[1][0].set_title("Gradient Boosting feature selection")
axs[1][0].bar(x_bar, PI_gradient_boosting_importance, width=width)
axs[1][0].bar(x_bar+width, to_distribution(gradient_boosting.feature_importances_), width=width)
axs[1][0].legend(["PI importances", "Model importances"])
axs[1][1].set_title("SVM feature selection")
axs[1][1].bar(x_bar, PI_svm_importance, width=width)
axs[1][1].bar(x_bar+width, to_distribution(svm.coef_.squeeze()), width=width)
axs[1][1].legend(["PI importances", "Model importances"])

plt.show()


In [ ]:
# Accuracies for each classifier

accuracies_filtered_PI = []
X_filtered_rf = np.copy(X)
X_filtered_lr = np.copy(X)
X_filtered_gb = np.copy(X)
X_filtered_svm = np.copy(X)

for i in trange(0, X.shape[-1]):

  accuracies_filtered_PI.append([
		(cross_val_score(random_forest, X_filtered_rf, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered_lr, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered_gb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered_svm, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])

  random_forest_filtered = RandomForestClassifier(random_state=0).fit(X_filtered_rf,y)
  logistic_regression_filtered = LogisticRegression(random_state=0).fit(X_filtered_lr,y)
  gradient_boosting_filtered = GradientBoostingClassifier(random_state=0).fit(X_filtered_gb,y)
  svm_filtered = SVC(kernel = "linear", random_state=0).fit(X_filtered_svm,y)

  PI_random_forest_filtered = permutation_importance(random_forest_filtered, X_filtered_rf, y, n_repeats=10, random_state=0)
  PI_logistic_regression_filtered = permutation_importance(logistic_regression_filtered, X_filtered_lr, y, n_repeats=10, random_state=0)
  PI_gradient_boosting_filtered = permutation_importance(gradient_boosting_filtered, X_filtered_gb, y, n_repeats=10, random_state=0)
  PI_svm_filtered = permutation_importance(svm_filtered, X_filtered_svm, y, n_repeats=10, random_state=0)

  PI_random_forest_filtered_feature_importance = to_distribution(PI_random_forest_filtered.importances_mean)
  PI_logistic_regression_filtered_feature_importance = to_distribution(PI_logistic_regression_filtered.importances_mean)
  PI_gradient_boosting_filtered_feature_importance = to_distribution(PI_gradient_boosting_filtered.importances_mean)
  PI_svm_filtered_feature_importance = to_distribution(PI_svm_filtered.importances_mean)

  X_filtered_rf = X_filtered_rf[:, np.argsort(PI_random_forest_filtered_feature_importance)[::-1][:-1]]
  X_filtered_lr = X_filtered_lr[:, np.argsort(PI_logistic_regression_filtered_feature_importance)[::-1][:-1]]
  X_filtered_gb = X_filtered_gb[:, np.argsort(PI_gradient_boosting_filtered_feature_importance)[::-1][:-1]]
  X_filtered_svm = X_filtered_svm[:, np.argsort(PI_svm_filtered_feature_importance)[::-1][:-1]]

In [ ]:
accuracies_filtered_singularly = np.array(accuracies_filtered_singularly)
accuracies_filtered_PI = np.array(accuracies_filtered_PI)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PI[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1]+1, 3))
plt.legend(["Singularly", "PI"])
plt.show()

In [ ]:
PI_random_forest_top_20_features = np.argsort(PI_random_forest_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_logistic_regression_top_20_features = np.argsort(PI_logistic_regression_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_gradient_boosting_top_20_features = np.argsort(PI_gradient_boosting_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_svm_top_20_features = np.argsort(PI_svm_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]

print(f"""
Accuracies:
Random Forest: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
Logistic Regression: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
Gradient Boosting: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
SVC: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
""")

In [ ]:
print('R F / R F with PI')
print(np.array((random_forest_top_20_features, PI_random_forest_top_20_features)).squeeze())
print('L R / L R with PI')
print(np.array((logistic_regression_top_20_features,PI_logistic_regression_top_20_features)).squeeze())
print('G B / G B with PI')
print(np.array((gradient_boosting_top_20_features,PI_gradient_boosting_top_20_features)).squeeze())
print('SVM / SVM with PI')
print(np.array((svm_top_20_features,PI_svm_top_20_features)).squeeze())

#### PI with other singular classifiers

In [ ]:
# Features importances plot for each classifier

random_forest = RandomForestClassifier(random_state=0).fit(X,y)
logistic_regression = LogisticRegression(random_state=0).fit(X,y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(X,y)
svm = SVC(kernel = "linear", random_state=0).fit(X,y)
knn = KNeighborsClassifier().fit(X,y)
cat_nb = CategoricalNB().fit(X,y)
line_da = LinearDiscriminantAnalysis().fit(X,y)
quad_da = QuadraticDiscriminantAnalysis().fit(X,y)
ada_boost = AdaBoostClassifier().fit(X,y)
hist_gb = HistGradientBoostingClassifier().fit(X,y)

PI_random_forest = permutation_importance(random_forest, X, y, n_repeats=10, random_state=0)
PI_logistic_regression = permutation_importance(logistic_regression, X, y, n_repeats=10, random_state=0)
PI_gradient_boosting = permutation_importance(gradient_boosting, X, y, n_repeats=10, random_state=0)
PI_svm = permutation_importance(svm, X, y, n_repeats=10, random_state=0)
PI_knn = permutation_importance(knn, X, y, n_repeats=10, random_state=0)
PI_cat_nb = permutation_importance(cat_nb, X, y, n_repeats=10, random_state=0)
PI_line_da = permutation_importance(line_da, X, y, n_repeats=10, random_state=0)
PI_quad_da = permutation_importance(quad_da, X, y, n_repeats=10, random_state=0)
PI_ada_boost = permutation_importance(ada_boost, X, y, n_repeats=10, random_state=0)
PI_hist_gb = permutation_importance(hist_gb, X, y, n_repeats=10, random_state=0)

PI_random_forest_importance = to_distribution(PI_random_forest.importances_mean)
PI_logistic_regression_importance = to_distribution(PI_logistic_regression.importances_mean)
PI_gradient_boosting_importance = to_distribution(PI_gradient_boosting.importances_mean)
PI_svm_importance = to_distribution(PI_svm.importances_mean)
PI_knn_importance = to_distribution(PI_knn.importances_mean)
PI_cat_nb_importance = to_distribution(PI_cat_nb.importances_mean)
PI_line_da_importance = to_distribution(PI_line_da.importances_mean)
PI_quad_da_importance = to_distribution(PI_quad_da.importances_mean)
PI_ada_boost_importance = to_distribution(PI_ada_boost.importances_mean)
PI_hist_gb_importance = to_distribution(PI_hist_gb.importances_mean)

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .08
plt.figure(figsize=(25,8))
plt.title("PI feature importances")
plt.bar(x_bar, PI_random_forest_importance, width=width)
plt.bar(x_bar+width, PI_logistic_regression_importance, width=width)
plt.bar(x_bar+2*width, PI_gradient_boosting_importance, width=width)
plt.bar(x_bar+3*width, PI_svm_importance, width=width)
plt.bar(x_bar+4*width, PI_knn_importance, width=width)
plt.bar(x_bar+5*width, PI_cat_nb_importance, width=width)
plt.bar(x_bar+6*width, PI_line_da_importance, width=width)
plt.bar(x_bar+7*width, PI_quad_da_importance, width=width)
plt.bar(x_bar+8*width, PI_ada_boost_importance, width=width)
plt.bar(x_bar+9*width, PI_hist_gb_importance, width=width)
plt.legend(["PI rf", "PI lr", "PI gb", "PI svm", "PI knn", "PI cat_nb", "PI line_da", "PI quad_da", "PI ada_boost", "PI hist_gb"])

plt.show()


In [ ]:
# Accuracies for each classifier 

accuracies_filtered_PI = []
X_filtered_rf = np.copy(X)
X_filtered_lr = np.copy(X)
X_filtered_gb = np.copy(X)
X_filtered_svm = np.copy(X)
X_filtered_knn = np.copy(X)
X_filtered_cat_nb = np.copy(X)
X_filtered_line_da = np.copy(X)
X_filtered_quad_da = np.copy(X)
X_filtered_ada_boost = np.copy(X)
X_filtered_hist_gb = np.copy(X)

for i in trange(0, X.shape[-1]):

  accuracies_filtered_PI.append([
		(cross_val_score(random_forest, X_filtered_rf, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered_lr, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered_gb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered_svm, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(knn, X_filtered_knn, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(cat_nb, X_filtered_cat_nb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(line_da, X_filtered_line_da, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(quad_da, X_filtered_quad_da, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(ada_boost, X_filtered_ada_boost, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
    (cross_val_score(hist_gb, X_filtered_hist_gb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2)
	])

  random_forest_filtered = RandomForestClassifier().fit(X_filtered_rf,y)
  logistic_regression_filtered = LogisticRegression().fit(X_filtered_lr,y)
  gradient_boosting_filtered = GradientBoostingClassifier().fit(X_filtered_gb,y)
  svm_filtered = SVC(kernel = "linear").fit(X_filtered_svm,y)
  knn_filtered = KNeighborsClassifier().fit(X_filtered_knn,y)
  cat_nb_filtered = CategoricalNB().fit(X_filtered_cat_nb,y)
  line_da_filtered = LinearDiscriminantAnalysis().fit(X_filtered_line_da,y)
  quad_da_filtered = QuadraticDiscriminantAnalysis().fit(X_filtered_quad_da,y)
  ada_boost_filtered = AdaBoostClassifier().fit(X_filtered_ada_boost,y)
  hist_gb_filtered = HistGradientBoostingClassifier().fit(X_filtered_hist_gb,y)

  PI_random_forest_filtered = permutation_importance(random_forest_filtered, X_filtered_rf, y, n_repeats=10, random_state=0)
  PI_logistic_regression_filtered = permutation_importance(logistic_regression_filtered, X_filtered_lr, y, n_repeats=10, random_state=0)
  PI_gradient_boosting_filtered = permutation_importance(gradient_boosting_filtered, X_filtered_gb, y, n_repeats=10, random_state=0)
  PI_svm_filtered = permutation_importance(svm_filtered, X_filtered_svm, y, n_repeats=10, random_state=0)
  PI_knn_filtered = permutation_importance(knn_filtered, X_filtered_knn, y, n_repeats=10, random_state=0)
  PI_cat_nb_filtered = permutation_importance(cat_nb_filtered, X_filtered_cat_nb, y, n_repeats=10, random_state=0)
  PI_line_da_filtered = permutation_importance(line_da_filtered, X_filtered_line_da, y, n_repeats=10, random_state=0)
  PI_quad_da_filtered = permutation_importance(quad_da_filtered, X_filtered_quad_da, y, n_repeats=10, random_state=0)
  PI_ada_boost_filtered = permutation_importance(ada_boost_filtered, X_filtered_ada_boost, y, n_repeats=10, random_state=0)
  PI_hist_gb_filtered = permutation_importance(hist_gb_filtered, X_filtered_hist_gb, y, n_repeats=10, random_state=0)

  PI_random_forest_filtered_feature_importance = to_distribution(PI_random_forest_filtered.importances_mean)
  PI_logistic_regression_filtered_feature_importance = to_distribution(PI_logistic_regression_filtered.importances_mean)
  PI_gradient_boosting_filtered_feature_importance = to_distribution(PI_gradient_boosting_filtered.importances_mean)
  PI_svm_filtered_feature_importance = to_distribution(PI_svm_filtered.importances_mean)
  PI_knn_filtered_feature_importance = to_distribution(PI_knn_filtered.importances_mean)
  PI_cat_nb_filtered_feature_importance = to_distribution(PI_cat_nb_filtered.importances_mean)
  PI_line_da_filtered_feature_importance = to_distribution(PI_line_da_filtered.importances_mean)
  PI_quad_da_filtered_feature_importance = to_distribution(PI_quad_da_filtered.importances_mean)
  PI_ada_boost_filtered_feature_importance = to_distribution(PI_ada_boost_filtered.importances_mean)
  PI_hist_gb_filtered_feature_importance = to_distribution(PI_hist_gb_filtered.importances_mean)

  X_filtered_rf = X_filtered_rf[:, np.argsort(PI_random_forest_filtered_feature_importance)[::-1][:-1]]
  X_filtered_lr = X_filtered_lr[:, np.argsort(PI_logistic_regression_filtered_feature_importance)[::-1][:-1]]
  X_filtered_gb = X_filtered_gb[:, np.argsort(PI_gradient_boosting_filtered_feature_importance)[::-1][:-1]]
  X_filtered_svm = X_filtered_svm[:, np.argsort(PI_svm_filtered_feature_importance)[::-1][:-1]]
  X_filtered_knn = X_filtered_knn[:, np.argsort(PI_knn_filtered_feature_importance)[::-1][:-1]]
  X_filtered_cat_nb = X_filtered_cat_nb[:, np.argsort(PI_cat_nb_filtered_feature_importance)[::-1][:-1]]
  X_filtered_line_da = X_filtered_line_da[:, np.argsort(PI_line_da_filtered_feature_importance)[::-1][:-1]]
  X_filtered_quad_da = X_filtered_quad_da[:, np.argsort(PI_quad_da_filtered_feature_importance)[::-1][:-1]]
  X_filtered_ada_boost = X_filtered_ada_boost[:, np.argsort(PI_ada_boost_filtered_feature_importance)[::-1][:-1]]
  X_filtered_hist_gb = X_filtered_hist_gb[:, np.argsort(PI_hist_gb_filtered_feature_importance)[::-1][:-1]]

In [ ]:
accuracies_filtered_PI = np.array(accuracies_filtered_PI)

plt.figure(figsize=(7,5))

for i in range(10):
	plt.plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PI[:,i][::-1])

plt.xticks(np.arange(1, X.shape[-1]+1, 4))
plt.legend(["PI rf", "PI lr", "PI gb", "PI svm", "PI knn", "PI cat_nb", "PI line_da", "PI quad_da", "PI ada_boost", "PI hist_gb"])
plt.show()

In [ ]:
PI_random_forest_top_20_features = np.argsort(PI_random_forest_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_logistic_regression_top_20_features = np.argsort(PI_logistic_regression_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_gradient_boosting_top_20_features = np.argsort(PI_gradient_boosting_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_svm_top_20_features = np.argsort(PI_svm_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_knn_top_20_features = np.argsort(PI_knn_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_cat_nb_top_20_features = np.argsort(PI_cat_nb_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_line_da_top_20_features = np.argsort(PI_line_da_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_quad_da_top_20_features = np.argsort(PI_quad_da_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_ada_boost_top_20_features = np.argsort(PI_ada_boost_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
PI_hist_gb_top_20_features = np.argsort(PI_hist_gb_importance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]

print(f"""
Accuracies:
Random Forest: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
Logistic Regression: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
Gradient Boosting: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
SVC: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
knn: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][4]).round(2)}%
cat_nb: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][5]).round(2)}%
line_da: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][6]).round(2)}%
quad_da: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][7]).round(2)}%
ada_boost: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][8]).round(2)}%
hist_gb: {(accuracies_filtered_PI[-int(len(accuracies_filtered_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][9]).round(2)}%
""")

In [ ]:
print('r f: ', PI_random_forest_top_20_features)
print('l r: ', PI_logistic_regression_top_20_features)
print('g b: ', PI_gradient_boosting_top_20_features)
print('svm: ', PI_svm_top_20_features)
print('knn: ', PI_knn_top_20_features)
print('cnb: ', PI_cat_nb_top_20_features)
print('lda: ', PI_line_da_top_20_features)
print('qda: ', PI_quad_da_top_20_features)
print('abc: ', PI_ada_boost_top_20_features)
print('hgb: ', PI_hist_gb_top_20_features)

### LOCO

In [ ]:
def LOCO(model, X, y):
	res = []
	baseline = cross_val_score(model, X, y, n_jobs=-1, cv=N_CV).mean()
	for i in range(X.shape[-1]):
		loco_acc = cross_val_score(model, X[:, np.where(np.arange(0, X.shape[-1]) != i)[0]], y, n_jobs=-1, cv=N_CV).mean()
		res.append(loco_acc - baseline)
	return np.array(res)
def LOCO_scores_to_importance(res):
	res = -res
	return res - np.min(res)

In [ ]:
%%time
np.random.seed(0)
random_forest = RandomForestClassifier(random_state=0)
plt.bar(np.arange(0, X.shape[-1]) , LOCO_scores_to_importance(LOCO(random_forest, X, y)))
plt.show()

In [ ]:
random_forest = RandomForestClassifier(random_state=0)
logistic_regression = LogisticRegression(random_state=0)
gradient_boosting = GradientBoostingClassifier(random_state=0)
svm = SVC(kernel = "linear",random_state=0)

In [ ]:
rf_lofoimportance = LOCO_scores_to_importance(LOCO(random_forest, X, y))
lr_lofoimportance = LOCO_scores_to_importance(LOCO(logistic_regression, X, y))
gb_lofoimportance = LOCO_scores_to_importance(LOCO(gradient_boosting, X, y))
svm_lofoimportance = LOCO_scores_to_importance(LOCO(svm, X, y))

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .2
_, axs = plt.subplots(2, 2, figsize=(20,10))
axs[0][0].set_title("Random forest feature selection")
axs[0][0].bar(x_bar,to_distribution(rf_lofoimportance), width=width)
random_forest.fit(X,y)
axs[0][0].bar(x_bar+width, to_distribution(random_forest.feature_importances_), width=width)
axs[0][0].legend(["LOFO importances", "Model importances"])
axs[0][1].set_title("Logistic regression feature selection")
axs[0][1].bar(x_bar, to_distribution(lr_lofoimportance), width=width)
logistic_regression.fit(X,y)
axs[0][1].bar(x_bar+width, to_distribution(logistic_regression.coef_.squeeze()), width=width)
axs[0][1].legend(["LOFO importances", "Model importances"])
axs[1][0].set_title("Gradient Boosting feature selection")
axs[1][0].bar(x_bar, to_distribution(gb_lofoimportance), width=width)
gradient_boosting.fit(X,y)
axs[1][0].bar(x_bar+width, to_distribution(gradient_boosting.feature_importances_), width=width)
axs[1][0].legend(["LOFO importances", "Model importances"])
axs[1][1].set_title("SVM feature selection")
axs[1][1].bar(x_bar, to_distribution(svm_lofoimportance), width=width)
svm.fit(X,y)
axs[1][1].bar(x_bar+width, to_distribution(svm.coef_.squeeze()), width=width)
axs[1][1].legend(["LOFO importances", "Model importances"])

plt.show()

In [ ]:
accuracies_filtered_LOFO= []
X_filtered_rf = np.copy(X)
X_filtered_lr=np.copy(X)
X_filtered_gb=np.copy(X)
X_filtered_svm=np.copy(X)
for i in trange(0, X.shape[-1]-1):
  accuracies_filtered_LOFO.append([
		(cross_val_score(random_forest, X_filtered_rf, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered_lr, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered_gb, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered_svm, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
  rf_filtered_lofo_importance = LOCO_scores_to_importance(LOCO(random_forest, X_filtered_rf, y))
  lr_filtered_lofo_importance = LOCO_scores_to_importance(LOCO(logistic_regression, X_filtered_lr, y))
  gb_filtered_lofo_importance = LOCO_scores_to_importance(LOCO(gradient_boosting, X_filtered_gb, y))
  svm_filtered_lofo_importance = LOCO_scores_to_importance(LOCO(svm, X_filtered_svm, y))

  lofo_random_forest_filtered_feature_importance = to_distribution(rf_filtered_lofo_importance)
  lofo_logistic_regression_filtered_feature_importance = to_distribution(lr_filtered_lofo_importance)
  lofo_gradient_boosting_filtered_feature_importance = to_distribution(gb_filtered_lofo_importance)
  lofo_svm_filtered_feature_importance = to_distribution(svm_filtered_lofo_importance)

  X_filtered_rf = X_filtered_rf[:, np.argsort(lofo_random_forest_filtered_feature_importance)[::-1][:-1]]
  X_filtered_lr = X_filtered_lr[:, np.argsort(lofo_logistic_regression_filtered_feature_importance)[::-1][:-1]]
  X_filtered_gb = X_filtered_gb[:, np.argsort( lofo_gradient_boosting_filtered_feature_importance)[::-1][:-1]]
  X_filtered_svm = X_filtered_svm[:, np.argsort(lofo_svm_filtered_feature_importance)[::-1][:-1]]

In [ ]:
accuracies_filtered_singularly = np.array(accuracies_filtered_singularly)
accuracies_filtered_LOFO = np.array(accuracies_filtered_LOFO)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(2, X.shape[-1]+1), accuracies_filtered_LOFO[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PI[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "LOFO", "PI"])

plt.show()

In [ ]:
LOFO_random_forest_top_20_features = np.argsort(rf_lofoimportance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
LOFO_logistic_regression_top_20_features = np.argsort(lr_lofoimportance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
LOFO_gradient_boosting_top_20_features=np.argsort(gb_lofoimportance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
LOFO_svm_top_20_features=np.argsort(svm_lofoimportance)[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]
print(f"""
Accuracies:
Random Forest: {(accuracies_filtered_LOFO[-int(len(accuracies_filtered_LOFO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
Logistic Regression: {(accuracies_filtered_LOFO[-int(len(accuracies_filtered_LOFO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
Gradient Boosting: {(accuracies_filtered_LOFO[-int(len(accuracies_filtered_LOFO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
SVC: {(accuracies_filtered_LOFO[-int(len(accuracies_filtered_LOFO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
""")

## FS with model independent without classifiers

here:
	- PCA (allenarlo su tutto vs allenarlo solo su quelli onesti)
	- SparsePCA
	- Factor analysis
  	- Relief
  	- Mutual Information
  	- Minimum Redundancy Maximum Relevance (mRMR)
  	- tutta la roba che c'è qua https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import FactorAnalysis

In [ ]:
def calc_loadings(X, X_trans):
	loadings = np.zeros(shape = (X_trans.shape[1], X.shape[1]))
	for i in range(X_trans.shape[1]):
		for j in range(X.shape[1]):
			loadings[i,j] = np.cov(X_trans[:,i], X[:,j])[0][1]
	return loadings
def loadings_to_importance(loadings):
	return np.abs(loadings).sum(axis=0)

### PCA

In [ ]:
# PCA without limits in number of components or explained variance
pca = PCA()
# Fitting on all the data
pca.fit(X)

plt.imshow(np.abs(calc_loadings(X, pca.transform(X))))
plt.colorbar()
plt.show()

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(X, y)
logistic_regression = LogisticRegression(random_state=0).fit(X, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(X, y)
svm = SVC(kernel="linear", random_state=0).fit(X, y)
summed_importances = np.sum([
	to_distribution(random_forest.feature_importances_),
	to_distribution(logistic_regression.coef_.squeeze()),
	to_distribution(gradient_boosting.feature_importances_),
	to_distribution(svm.coef_.squeeze())
], axis=0)

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .2
plt.figure(dpi=200, figsize=(20,5))
plt.bar(x_bar, to_distribution(loadings_to_importance(calc_loadings(X, pca.transform(X)))), width=width)
plt.bar(x_bar+width, to_distribution(summed_importances), width=width)
plt.legend(["PCA importances", "summed importances"])
plt.show()

In [ ]:
accuracies_filtered_PCA_BS = []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_PCA_BS.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	pca = PCA()
	pca.fit(X_filtered)
	loadings = calc_loadings(X_filtered, pca.transform(X_filtered))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_PCA = []
X_filtered = np.copy(X)
pca = PCA()
pca.fit(X_filtered)
loadings = calc_loadings(X_filtered, pca.transform(X_filtered))
importances = loadings_to_importance(loadings)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_PCA.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])

	X_filtered = X[:, np.argsort(to_distribution(importances))[::-1][:-(i+1)]]

In [ ]:
accuracies_filtered_singularly = np.array(accuracies_filtered_singularly)
accuracies_filtered_PCA_BS = np.array(accuracies_filtered_PCA_BS)
accuracies_filtered_PCA = np.array(accuracies_filtered_PCA)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PCA[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PCA_BS[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "PCA", "PCA BS"])
plt.show()

### Kernel PCA

In [ ]:
# KernelPCA without limits in number of components or explained variance
kernelpca = KernelPCA(kernel="cosine")
# Fitting on all the data
kernelpca.fit(X)
plt.imshow(np.abs(calc_loadings(X, kernelpca.transform(X))))
plt.colorbar()
plt.show()

In [ ]:
accuracies_filtered_KPCA = []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_KPCA.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	kpca = KernelPCA(kernel="cosine")
	kpca.fit(X_filtered)
	loadings = calc_loadings(X_filtered, kpca.transform(X_filtered))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_KPCA_honest = []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_KPCA_honest.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	kpca = KernelPCA(kernel="cosine")
	kpca.fit(X_filtered[y==1])
	loadings = calc_loadings(X_filtered[y==1], kpca.transform(X_filtered[y==1]))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_KPCA_honest = np.array(accuracies_filtered_KPCA_honest)
accuracies_filtered_KPCA = np.array(accuracies_filtered_KPCA)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_KPCA[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_KPCA_honest[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "KPCA", "KPCA honest only"])
plt.show()

### Sparse PCA

In [ ]:
# SparsePCA without limits in number of components or explained variance
sparsepca = SparsePCA()
# Fitting on all the data
sparsepca.fit(X)
plt.imshow(np.abs(calc_loadings(X, sparsepca.transform(X))))
plt.colorbar()
plt.show()

In [ ]:
accuracies_filtered_SparsePCA = []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_SparsePCA.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	sparse_pca = SparsePCA()
	sparse_pca.fit(X_filtered)
	loadings = calc_loadings(X_filtered, sparse_pca.transform(X_filtered))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_SparsePCA = np.array(accuracies_filtered_SparsePCA)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_KPCA[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PCA_BS[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_SparsePCA[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "KPCA", "PCA BS", "SparsePCA"])
plt.show()

### Truncated SVD

In [ ]:
# TruncatedSVD without limits in number of components or explained variance
truncatedsvd = TruncatedSVD(n_components = X.shape[1]-1)
# Fitting on all the data
truncatedsvd.fit(X)
plt.imshow(np.abs(calc_loadings(X, truncatedsvd.transform(X))))
plt.colorbar()
plt.show()

In [ ]:
accuracies_filtered_trunc_SVD = []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]-1):
	accuracies_filtered_trunc_SVD.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	truncated_svd = TruncatedSVD()
	truncated_svd.fit(X_filtered)
	loadings = calc_loadings(X_filtered, truncated_svd.transform(X_filtered))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_trunc_SVD = np.array(accuracies_filtered_trunc_SVD)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_KPCA[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PCA_BS[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_SparsePCA[:,i][::-1])
	axs[i].plot(np.arange(2, X.shape[-1]+1), accuracies_filtered_trunc_SVD[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "KPCA", "PCA BS", "SparsePCA", "TruncatedSVD"])
plt.show()

### Factor Analysis

In [ ]:
# FastICA without limits in number of components or explained variance
factor = FactorAnalysis()
# Fitting on all the data
factor.fit(X)
plt.imshow(np.abs(calc_loadings(X, factor.transform(X))))
plt.colorbar()
plt.show()

In [ ]:
accuracies_filtered_factor_analysis= []
X_filtered = np.copy(X)

for i in trange(0, X.shape[-1]):
	accuracies_filtered_factor_analysis.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])
	factor_analysis = FactorAnalysis()
	factor_analysis.fit(X_filtered)
	loadings = calc_loadings(X_filtered, factor_analysis.transform(X_filtered))
	importances = loadings_to_importance(loadings)

	X_filtered = X_filtered[:, np.argsort(to_distribution(importances))[::-1][:-1]]

In [ ]:
accuracies_filtered_factor_analysis = np.array(accuracies_filtered_factor_analysis)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_KPCA[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_PCA_BS[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_SparsePCA[:,i][::-1])
	axs[i].plot(np.arange(2, X.shape[-1]+1), accuracies_filtered_trunc_SVD[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_factor_analysis[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1], 3))
plt.legend(["Singularly", "KPCA", "PCA BS", "SparsePCA", "TruncatedSVD", "Factor Analysis"])
plt.show()

### Relief

In [ ]:
#!pip install skrebate

In [ ]:
from skrebate import ReliefF

In [ ]:
relief = ReliefF(n_neighbors=5).fit(X,y)

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(X,y)
logistic_regression = LogisticRegression(random_state=0).fit(X,y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(X,y)
svm = SVC(kernel = "linear", random_state=0).fit(X,y)

In [ ]:
summed_importances=np.sum([
	to_distribution(random_forest.feature_importances_),
	to_distribution(logistic_regression.coef_.squeeze()),
	to_distribution(gradient_boosting.feature_importances_),
	to_distribution( svm.coef_.squeeze())
], axis=0)

In [ ]:
x_bar = np.arange(1, X.shape[-1]+1)
width = .2
plt.figure(dpi=200, figsize=(20,5))
plt.bar(x_bar, to_distribution(relief.feature_importances_), width=width)
plt.bar(x_bar+width, to_distribution(summed_importances), width=width)
plt.legend(["Relief importances", "Summed importances"])
plt.show()

In [ ]:
accuracies_with_relief_selection= []
X_filtered = np.copy(X)
for i in trange(0, X.shape[-1]):
	accuracies_with_relief_selection.append([
		(cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
	])

	relief = ReliefF(n_neighbors=5).fit(X_filtered,y)
	relief_feature_importance=to_distribution(relief.feature_importances_)
	X_filtered= X_filtered[:, np.argsort(relief_feature_importance)[::-1][:-1]]

In [ ]:
_, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].set_title("Individual feature selection")
axs[0].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[::-1])
axs[0].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[0].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[0].set_xlabel("Number of top feature considered in the training")
axs[0].set_ylabel("Accuracy")

axs[1].set_title("Relief feature selection")
axs[1].plot(np.arange(1, X.shape[-1]+1), accuracies_with_relief_selection[::-1])
axs[1].set_xticks(np.arange(1, X.shape[-1]+1, 3))
axs[1].legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
axs[1].set_xlabel("Number of top feature considered in the training")
axs[1].set_ylabel("Accuracy")

plt.show()

In [ ]:
accuracies_filtered_singularly = np.array(accuracies_filtered_singularly)
accuracies_with_relief_selection = np.array(accuracies_with_relief_selection)

_, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_singularly[:,i][::-1])
	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_with_relief_selection[:,i][::-1])
	axs[i].set_xticks(np.arange(1, X.shape[-1]+1, 3))
plt.legend(["Singularly", "Relief"])
plt.show()

In [ ]:
relief_top_20_features = np.argsort(to_distribution(relief.feature_importances_))[::-1][:int(original_dataset_size * (1-PERCENTAGE_FEATURE_TO_REMOVE))]

print(f"""
Accuracies:
Random Forest: {(accuracies_with_relief_selection[-int(len(accuracies_with_relief_selection) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
Logistic Regression: {(accuracies_with_relief_selection[-int(len(accuracies_with_relief_selection) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
Gradient Boosting: {(accuracies_with_relief_selection[-int(len(accuracies_with_relief_selection) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
SVC: {(accuracies_with_relief_selection[-int(len(accuracies_with_relief_selection) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
""")

## Robustness of methods

### Robustness Indexes

In [ ]:
from itertools import combinations
def leaderboards_to_importance(elms):
	elms = np.array([
		l.argsort()
		for l in elms
	])
	return np.mean([
		np.linalg.norm(elms[c[0]] - elms[c[1]],ord=1)
		for c in combinations(list(range(len(elms))), 2)
	])

In [ ]:
example = np.array([
	[4,2,3,1,0],
	[4,3,1,2,0],
	[4,3,2,1,0],
	[4,3,2,1,0]
])


In [ ]:
leaderboards_to_importance(example)

### Permutation Importance

In [ ]:
accuracies_filtered_together_PI = []
X_filtered = np.copy(X)
feature_importance_PI = []
features_selected_at_each_step_PI = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_PI = []
for i in trange(X.shape[-1], 0, -1):
	accuracies_filtered_together_PI.append([
		# (cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(knn, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(cat_nb, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(line_da, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(quad_da, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(ada_boost, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(hist_gb, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2)
	])
	if i > 1 :
		# random_forest=RandomForestClassifier(random_state=0).fit(X_filtered, y)
		logistic_regression=LogisticRegression(random_state=0).fit(X_filtered, y)
		# gradient_boosting=GradientBoostingClassifier(random_state=0).fit(X_filtered, y)
		svm = SVC(kernel = "linear",random_state=0).fit(X_filtered, y)
		# knn = KNeighborsClassifier().fit(X_filtered,y)
		cat_nb = CategoricalNB().fit(X_filtered,y)
		line_da = LinearDiscriminantAnalysis().fit(X_filtered,y)
		# quad_da = QuadraticDiscriminantAnalysis().fit(X_filtered,y)
		# ada_boost = AdaBoostClassifier().fit(X_filtered,y)
		hist_gb = HistGradientBoostingClassifier().fit(X_filtered,y)

		# random_forest_feature_importance_PI = to_distribution(permutation_importance(random_forest, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		logistic_regression_feature_importance_PI = to_distribution(permutation_importance(logistic_regression, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# gradient_boosting_feature_importance_PI = to_distribution(permutation_importance(gradient_boosting, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		svm_feature_importance_PI = to_distribution(permutation_importance(svm, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# knn_feature_importance_PI = to_distribution(permutation_importance(knn, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		cat_nb_feature_importance_PI = to_distribution(permutation_importance(cat_nb, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		line_da_feature_importance_PI = to_distribution(permutation_importance(line_da, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# quad_da_feature_importance_PI = to_distribution(permutation_importance(quad_da, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# ada_boost_feature_importance_PI = to_distribution(permutation_importance(ada_boost, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		hist_gb_feature_importance_PI = to_distribution(permutation_importance(hist_gb, X_filtered, y, n_repeats=10, random_state=0).importances_mean)

		summed_importance_PI = np.sum([
			# random_forest_feature_importance_PI,
			logistic_regression_feature_importance_PI,
			# gradient_boosting_feature_importance_PI,
			svm_feature_importance_PI,
			# knn_feature_importance_PI,
			cat_nb_feature_importance_PI,
			line_da_feature_importance_PI,
			# quad_da_feature_importance_PI,
			# ada_boost_feature_importance_PI,
			hist_gb_feature_importance_PI
		], axis=0)

		feature_importance_PI.append(summed_importance_PI)
		selected_features_PI = np.argsort(summed_importance_PI)[::-1][:-1]
		features_selected_at_each_step_PI.append(selected_features_PI)
		features_importance_at_each_step_PI.append([
			# random_forest_feature_importance_PI,
			logistic_regression_feature_importance_PI,
			# gradient_boosting_feature_importance_PI,
			svm_feature_importance_PI,
			# knn_feature_importance_PI,
			cat_nb_feature_importance_PI,
			line_da_feature_importance_PI,
			# quad_da_feature_importance_PI,
			# ada_boost_feature_importance_PI,
			hist_gb_feature_importance_PI
		])

		X_filtered = X_filtered[:,selected_features_PI]

In [ ]:
# plt.plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together_PI[::-1])
# plt.xticks(np.arange(1, X.shape[-1]+1, 4))

# plt.legend(["Random Forest",
# "Logistic Regression",
# "Gradient Boosting",
# "SVM",
# "k Nearest Neighbors",
# "Nayve Bayes",
# "Linear Discriminant Analysis",
# "Quadratic Discriminnt Analysis",
# "AdaBoost",
# "histogram-based Gradient Boosting"])

# plt.xlabel("Number of top feature considered in the training")
# plt.ylabel("Accuracy")
# plt.show()

In [ ]:
# print(f"""
# Accuracies:
# Random Forest: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
# Logistic Regression: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
# Gradient Boosting: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
# SVM: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
# k Nearest Neighbors: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][4]).round(2)}%
# Nayve Bayes: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][5]).round(2)}%
# Linear Discriminant Analysis: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][6]).round(2)}%
# Quadratic Discriminnt Analysis: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][7]).round(2)}%
# AdaBoost: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][8]).round(2)}%
# histogram-based Gradient Boosting: {(accuracies_filtered_together_PI[-int(len(accuracies_filtered_together_PI) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][9]).round(2)}%
# """)

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_PI[i]]
new_X_PI = X[:,indexes]

In [ ]:
# random_forest = RandomForestClassifier(random_state=0).fit(new_X_PI, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_PI, y)
# gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_PI, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_PI, y)
# knn = KNeighborsClassifier().fit(new_X_PI,y)
cat_nb = CategoricalNB().fit(new_X_PI,y)
line_da = LinearDiscriminantAnalysis().fit(new_X_PI,y)
# quad_da = QuadraticDiscriminantAnalysis().fit(new_X_PI,y)
# ada_boost = AdaBoostClassifier().fit(new_X_PI,y)
hist_gb = HistGradientBoostingClassifier().fit(new_X_PI,y)


In [ ]:
# importances for each model for the best features
# PI_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
PI_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
# PI_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
PI_svm_importances = np.argsort(permutation_importance(svm, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
# PI_knn_importances = np.argsort(permutation_importance(knn, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
PI_cat_nb_importances = np.argsort(permutation_importance(cat_nb, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
PI_line_da_importances = np.argsort(permutation_importance(line_da, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
# PI_quad_da_importances = np.argsort(permutation_importance(quad_da, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
# PI_ada_boost_importances = np.argsort(permutation_importance(ada_boost, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)
PI_hist_gb_importances = np.argsort(permutation_importance(hist_gb, new_X_PI, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
# soliti 4 classifiers
# array([[2, 4, 3, 1, 0],
#        [4, 2, 3, 1, 0],
#        [3, 4, 1, 2, 0],
#        [4, 2, 3, 1, 0]])

# tutti 10 (no knn)
# array([[4, 2, 3, 1, 0],
#        [4, 2, 3, 1, 0],
#        [3, 4, 1, 2, 0],
#        [4, 2, 3, 1, 0],
#        [4, 2, 3, 1, 0],
#        [4, 2, 3, 1, 0],
#        [3, 4, 1, 2, 0],
#        [2, 4, 3, 0, 1],
#        [4, 2, 1, 3, 0]])

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost)
# array([[3, 4, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0]])

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost, random_forest)
# array([[4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0]])




# soliti 4 classifiers
# r i = 3.666
# acc = 74.072

# tutti 10 (no knn)
# r i = 3.444
# acc = 74.668

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost)
# r i = 0.667
# acc = 73.623

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost, random_forest)
# r i = 0.000
# acc = 73.532

In [ ]:
PI_lead = np.array([
	# PI_random_forest_importances,
	PI_logistic_regression_importances,
	# PI_gradient_boosting_importances,
	PI_svm_importances,
	# PI_knn_importances,
	PI_cat_nb_importances,
	PI_line_da_importances,
	# PI_quad_da_importances,
	# PI_ada_boost_importances,
	PI_hist_gb_importances
])
PI_lead

In [ ]:
# robustness index
PI_robustness_score = leaderboards_to_importance(PI_lead)
PI_robustness_score

In [ ]:
#  PI accuracy
index = int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))
PI_accuracy = np.array(accuracies_filtered_together_PI)[index].mean()
PI_accuracy

### LOCO

In [ ]:
accuracies_filtered_together_LOCO = []
X_filtered = np.copy(X)
feature_importance_LOCO = []
features_selected_at_each_step_LOCO= [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_LOCO = []
for i in trange(X.shape[-1], 0, -1):
	accuracies_filtered_together_LOCO.append([
		# (cross_val_score(random_forest, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(logistic_regression, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(gradient_boosting, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(svm, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(knn, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(cat_nb, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(line_da, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(quad_da, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		# (cross_val_score(ada_boost, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2),
		(cross_val_score(hist_gb, X_filtered, y, cv=N_CV, n_jobs=-1).mean()*100).round(2)
	])
	if i > 1 :
		# random_forest=RandomForestClassifier(random_state=0).fit(X_filtered, y)
		logistic_regression=LogisticRegression(random_state=0).fit(X_filtered, y)
		# gradient_boosting=GradientBoostingClassifier(random_state=0).fit(X_filtered, y)
		svm = SVC(kernel = "linear",random_state=0).fit(X_filtered, y)
		# knn = KNeighborsClassifier().fit(X_filtered,y)
		cat_nb = CategoricalNB().fit(X_filtered,y)
		line_da = LinearDiscriminantAnalysis().fit(X_filtered,y)
		# quad_da = QuadraticDiscriminantAnalysis().fit(X_filtered,y)
		# ada_boost = AdaBoostClassifier().fit(X_filtered,y)
		hist_gb = HistGradientBoostingClassifier().fit(X_filtered,y)

		# random_forest_feature_importance_LOCO = to_distribution(permutation_importance(random_forest, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		logistic_regression_feature_importance_LOCO = to_distribution(permutation_importance(logistic_regression, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# gradient_boosting_feature_importance_LOCO = to_distribution(permutation_importance(gradient_boosting, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		svm_feature_importance_LOCO = to_distribution(permutation_importance(svm, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# knn_feature_importance_LOCO = to_distribution(permutation_importance(knn, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		cat_nb_feature_importance_LOCO = to_distribution(permutation_importance(cat_nb, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		line_da_feature_importance_LOCO = to_distribution(permutation_importance(line_da, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# quad_da_feature_importance_LOCO = to_distribution(permutation_importance(quad_da, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		# ada_boost_feature_importance_LOCO = to_distribution(permutation_importance(ada_boost, X_filtered, y, n_repeats=10, random_state=0).importances_mean)
		hist_gb_feature_importance_LOCO = to_distribution(permutation_importance(hist_gb, X_filtered, y, n_repeats=10, random_state=0).importances_mean)

		summed_importance_LOCO = np.sum([
			# random_forest_feature_importance_LOCO,
			logistic_regression_feature_importance_LOCO,
			# gradient_boosting_feature_importance_LOCO,
			svm_feature_importance_LOCO,
			# knn_feature_importance_LOCO,
			cat_nb_feature_importance_LOCO,
			line_da_feature_importance_LOCO,
			# quad_da_feature_importance_LOCO,
			# ada_boost_feature_importance_LOCO,
			hist_gb_feature_importance_LOCO
		], axis=0)

		feature_importance_LOCO.append(summed_importance_LOCO)
		selected_features_LOCO = np.argsort(summed_importance_LOCO)[::-1][:-1]
		features_selected_at_each_step_LOCO.append(selected_features_LOCO)
		features_importance_at_each_step_LOCO.append([
			# random_forest_feature_importance_LOCO,
			logistic_regression_feature_importance_LOCO,
			# gradient_boosting_feature_importance_LOCO,
			svm_feature_importance_LOCO,
			# knn_feature_importance_LOCO,
			cat_nb_feature_importance_LOCO,
			line_da_feature_importance_LOCO,
			# quad_da_feature_importance_LOCO,
			# ada_boost_feature_importance_LOCO,
			hist_gb_feature_importance_LOCO
		])

		X_filtered = X_filtered[:,selected_features_LOCO]

In [ ]:
# plt.plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together_LOCO[::-1])
# plt.xticks(np.arange(1, X.shape[-1]+1, 3))
# plt.legend(["Random forest", "Logistic regression", "Gradient boosting", "SVM"])
# plt.xlabel("Number of top feature considered in the training")
# plt.ylabel("Accuracy")
# plt.show()

In [ ]:
# accuracies_filtered_LOFO= np.array(accuracies_filtered_LOFO)
# accuracies_filtered_together_LOCO=np.array(accuracies_filtered_together_LOCO)
# _, axs = plt.subplots(1, 4, figsize=(15,5))
# for i in range(4):
# 	axs[i].plot(np.arange(1, X.shape[-1]+1), accuracies_filtered_together_LOCO[:,i][::-1])
# 	axs[i].plot(np.arange(2, X.shape[-1]+1), accuracies_filtered_LOFO[:,i][::-1])
# 	axs[i].set_xticks(np.arange(1, X.shape[-1]+1, 3))
# plt.legend(["Together", "Singularly"])
# plt.show()

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_LOCO[i]]
new_X_LOCO = X[:,indexes]

In [ ]:
# random_forest = RandomForestClassifier(random_state=0).fit(new_X_LOCO, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_LOCO, y)
# gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_LOCO, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_LOCO, y)
# knn = KNeighborsClassifier().fit(new_X_LOCO,y)
cat_nb = CategoricalNB().fit(new_X_LOCO,y)
line_da = LinearDiscriminantAnalysis().fit(new_X_LOCO,y)
# quad_da = QuadraticDiscriminantAnalysis().fit(new_X_LOCO,y)
# ada_boost = AdaBoostClassifier().fit(new_X_LOCO,y)
hist_gb = HistGradientBoostingClassifier().fit(new_X_LOCO,y)

In [ ]:
# importances for each model for the best features
# LOCO_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
LOCO_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
# LOCO_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
LOCO_svm_importances = np.argsort(permutation_importance(svm, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
# LOCO_knn_importances = np.argsort(permutation_importance(knn, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
LOCO_cat_nb_importances = np.argsort(permutation_importance(cat_nb, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
LOCO_line_da_importances = np.argsort(permutation_importance(line_da, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
# LOCO_quad_da_importances = np.argsort(permutation_importance(quad_da, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
# LOCO_ada_boost_importances = np.argsort(permutation_importance(ada_boost, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)
LOCO_hist_gb_importances = np.argsort(permutation_importance(hist_gb, new_X_LOCO, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
# soliti 4 classifiers
# array([[4, 2, 3, 1, 0],
#        [4, 2, 3, 1, 0],
#        [3, 4, 1, 2, 0],
#        [4, 2, 3, 1, 0]]

# tutti 10
# array([[4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [2, 4, 1, 3, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [2, 4, 1, 3, 0],
#        [3, 4, 2, 0, 1],
#        [4, 3, 1, 2, 0]])

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost, random_forest)
# array([[4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0],
#        [4, 3, 2, 1, 0]])




# soliti 4 classifiers
# r i = 3.000
# acc = 75.210

# tutti 10
# r i = 3.156
# acc = 74.545

# tutti 10 (no knn, gradient_boosting, quad_da, ada_boost, random_forest)
# r i = 0.000
# acc = 73.532

In [ ]:
# print(f"""
# Accuracies:
# Random Forest: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][0]).round(2)}%
# Logistic Regression: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][1]).round(2)}%
# Gradient Boosting: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][2]).round(2)}%
# SVM: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][3]).round(2)}%
# k Nearest Neighbors: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][4]).round(2)}%
# Nayve Bayes: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][5]).round(2)}%
# Linear Discriminant Analysis: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][6]).round(2)}%
# Quadratic Discriminnt Analysis: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][7]).round(2)}%
# AdaBoost: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][8]).round(2)}%
# histogram-based Gradient Boosting: {(accuracies_filtered_together_LOCO[-int(len(accuracies_filtered_together_LOCO) * (1-PERCENTAGE_FEATURE_TO_REMOVE))][9]).round(2)}%
# """)

In [ ]:
LOCO_lead = np.array([
	# LOCO_random_forest_importances,
	LOCO_logistic_regression_importances,
	# LOCO_gradient_boosting_importances,
	LOCO_svm_importances,
	# LOCO_knn_importances,
	LOCO_cat_nb_importances,
	LOCO_line_da_importances,
	# LOCO_quad_da_importances,
	# LOCO_ada_boost_importances,
	LOCO_hist_gb_importances
])
LOCO_lead

In [ ]:
# robustness index
LOCO_robustness_score = leaderboards_to_importance(LOCO_lead)
LOCO_robustness_score

In [ ]:
#  LOCO accuracy
index = int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))
LOCO_accuracy = np.array(accuracies_filtered_together_LOCO)[index].mean()
LOCO_accuracy

In [ ]:
print('Permutation Importance')
print('r i : ', PI_robustness_score)
print('acc : ', PI_accuracy)
print('Leave one Covariate Out')
print('r i : ', LOCO_robustness_score)
print('acc : ', LOCO_accuracy)

### PCA

In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_PCA = []
features_selected_at_each_step_PCA = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_PCA = []
for i in trange(X.shape[-1], 0, -1):
  if i > 1 :
    pca = PCA()
    pca.fit(X_filtered)
    loadings_PCA = calc_loadings(X_filtered, pca.transform(X_filtered))
    importances_PCA = to_distribution(loadings_to_importance(loadings_PCA))
    feature_importance_PCA.append(importances_PCA)
    selected_features_PCA = np.argsort(importances_PCA)[::-1][:-1]
    features_selected_at_each_step_PCA.append(selected_features_PCA)
    X_filtered = X_filtered[:, selected_features_PCA]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_PCA[i]]
new_X_PCA = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_PCA, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_PCA, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_PCA, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_PCA, y)

In [ ]:
##importances for each model for the best features
PCA_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_PCA, y, n_repeats=10, random_state=0).importances_mean)
PCA_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_PCA, y, n_repeats=10, random_state=0).importances_mean)
PCA_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_PCA, y, n_repeats=10, random_state=0).importances_mean)
PCA_svm_importancess = np.argsort(permutation_importance(svm, new_X_PCA, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
PCA_lead = np.array([
	PCA_random_forest_importances,
	PCA_logistic_regression_importances,
	PCA_gradient_boosting_importances,
	PCA_svm_importancess
])
PCA_lead

In [ ]:
PCA_robustness_score = leaderboards_to_importance(PCA_lead)
PCA_robustness_score

### KernelPCA


In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_KPCA = []
features_selected_at_each_step_KPCA = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_KPCA = []
for i in trange(X.shape[-1], 0, -1):
  if i > 1 :
    kpca = KernelPCA(kernel="cosine")
    kpca.fit(X_filtered)
    loadings_KPCA = calc_loadings(X_filtered, kpca.transform(X_filtered))
    importances_KPCA = to_distribution(loadings_to_importance(loadings_KPCA))
    feature_importance_KPCA.append(importances_KPCA)
    selected_features_KPCA = np.argsort(importances_KPCA)[::-1][:-1]
    features_selected_at_each_step_KPCA.append(selected_features_KPCA)
    X_filtered = X_filtered[:, selected_features_KPCA]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_KPCA[i]]
new_X_KPCA = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_KPCA, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_KPCA, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_KPCA, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_KPCA, y)

In [ ]:
##importances for each model for the best features
KPCA_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_KPCA, y, n_repeats=10, random_state=0).importances_mean)
KPCA_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_KPCA, y, n_repeats=10, random_state=0).importances_mean)
KPCA_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_KPCA, y, n_repeats=10, random_state=0).importances_mean)
KPCA_svm_importances = np.argsort(permutation_importance(svm, new_X_KPCA, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
KPCA_lead = np.array([
	KPCA_random_forest_importances,
	KPCA_logistic_regression_importances,
	KPCA_gradient_boosting_importances,
	KPCA_svm_importances
])
KPCA_lead

In [ ]:
KPCA_robustness_score = leaderboards_to_importance(KPCA_lead)
KPCA_robustness_score

### Sparse PCA

In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_SPCA = []
features_selected_at_each_step_SPCA = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_SPCA = []
for i in trange(X.shape[-1], 0, -1):
  if i > 1 :
    sparse_pca = SparsePCA()
    sparse_pca.fit(X_filtered)
    loadings_SPCA = calc_loadings(X_filtered, sparse_pca.transform(X_filtered))
    importances_SPCA = to_distribution(loadings_to_importance(loadings_SPCA))
    feature_importance_SPCA.append(importances_SPCA)
    selected_features_SPCA = np.argsort(importances_SPCA)[::-1][:-1]
    features_selected_at_each_step_SPCA.append(selected_features_SPCA)
    X_filtered = X_filtered[:, selected_features_SPCA]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_SPCA[i]]
new_X_SPCA = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_SPCA, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_SPCA, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_SPCA, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_SPCA, y)

In [ ]:
#features importances
SPCA_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_SPCA, y, n_repeats=10, random_state=0).importances_mean)
SPCA_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_SPCA, y, n_repeats=10, random_state=0).importances_mean)
SPCA_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_SPCA, y, n_repeats=10, random_state=0).importances_mean)
SPCA_svm_importances = np.argsort(permutation_importance(svm, new_X_SPCA, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
SPCA_lead = np.array([
	SPCA_random_forest_importances,
	SPCA_logistic_regression_importances,
	SPCA_gradient_boosting_importances,
	SPCA_svm_importances
])
SPCA_lead

In [ ]:
SPCA_robustness_score = leaderboards_to_importance(SPCA_lead)
SPCA_robustness_score

### truncated SVD

In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_TSVD = []
features_selected_at_each_step_TSVD = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_TSVD = []
for i in trange(X.shape[-1]-1, 0, -1):
  if i > 1 :
    truncated_svd = TruncatedSVD()
    truncated_svd.fit(X_filtered)
    loadings_TSVD = calc_loadings(X_filtered, truncated_svd.transform(X_filtered))
    importances_TSVD = to_distribution(loadings_to_importance(loadings_TSVD))
    feature_importance_TSVD.append(importances_TSVD)
    selected_features_TSVD = np.argsort(importances_TSVD)[::-1][:-1]
    features_selected_at_each_step_TSVD.append(selected_features_TSVD)
    X_filtered = X_filtered[:, selected_features_TSVD]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_TSVD[i]]
new_X_TSVD = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_TSVD, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_TSVD, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_TSVD, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_TSVD, y)

In [ ]:
#features importances
TSVD_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_TSVD, y, n_repeats=10, random_state=0).importances_mean)
TSVD_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_TSVD, y, n_repeats=10, random_state=0).importances_mean)
TSVD_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_TSVD, y, n_repeats=10, random_state=0).importances_mean)
TSVD_svm_importances = np.argsort(permutation_importance(svm, new_X_TSVD, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
TSVD_lead = np.array([
	TSVD_random_forest_importances,
	TSVD_logistic_regression_importances,
	TSVD_gradient_boosting_importances,
	TSVD_svm_importances
])
TSVD_lead

In [ ]:
TSVD_robustness_score = leaderboards_to_importance(TSVD_lead)
TSVD_robustness_score

### Factor analysis

In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_FA = []
features_selected_at_each_step_FA = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_FA = []
for i in trange(X.shape[-1]-1, 0, -1):
  if i > 1 :
    factor_analysis = FactorAnalysis()
    factor_analysis.fit(X_filtered)
    loadings_FA = calc_loadings(X_filtered, factor_analysis.transform(X_filtered))
    importances_FA = to_distribution(loadings_to_importance(loadings_FA))
    feature_importance_FA.append(importances_FA)
    selected_features_FA = np.argsort(importances_FA)[::-1][:-1]
    features_selected_at_each_step_FA.append(selected_features_FA)
    X_filtered = X_filtered[:, selected_features_FA]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_FA[i]]
new_X_FA = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_FA, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_FA, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_FA, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_FA, y)

In [ ]:
#features importances
FA_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_FA, y, n_repeats=10, random_state=0).importances_mean)
FA_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_FA, y, n_repeats=10, random_state=0).importances_mean)
FA_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_FA, y, n_repeats=10, random_state=0).importances_mean)
FA_svm_importances = np.argsort(permutation_importance(svm, new_X_FA, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
FA_lead = np.array([
	FA_random_forest_importances,
	FA_logistic_regression_importances,
	FA_gradient_boosting_importances,
	FA_svm_importances
])
FA_lead

In [ ]:
FA_robustness_score = leaderboards_to_importance(FA_lead)
FA_robustness_score

### Relief

In [ ]:
#features selected at each step
X_filtered = np.copy(X)
feature_importance_R = []
features_selected_at_each_step_R = [np.arange(0, X.shape[-1]).tolist()]
features_importance_at_each_step_R = []
for i in trange(X.shape[-1], 0, -1):
  if i > 1 :
    relief = ReliefF(n_neighbors=5).fit(X_filtered,y)
    importances_R=to_distribution(relief.feature_importances_)
    feature_importance_R.append(importances_R)
    selected_features_R = np.argsort(importances_R)[::-1][:-1]
    features_selected_at_each_step_R.append(selected_features_R)
    X_filtered = X_filtered[:, selected_features_R]

In [ ]:
indexes = np.arange(0, X.shape[-1])
for i in range(int(np.round(int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))) + 1)):
	indexes = indexes[features_selected_at_each_step_R[i]]
new_X_R = X[:,indexes]

In [ ]:
random_forest = RandomForestClassifier(random_state=0).fit(new_X_R, y)
logistic_regression = LogisticRegression(random_state=0).fit(new_X_R, y)
gradient_boosting = GradientBoostingClassifier(random_state=0).fit(new_X_R, y)
svm = SVC(kernel = "linear",random_state=0).fit(new_X_R, y)

In [ ]:
#features importances
R_random_forest_importances = np.argsort(permutation_importance(random_forest, new_X_R, y, n_repeats=10, random_state=0).importances_mean)
R_logistic_regression_importances = np.argsort(permutation_importance(logistic_regression, new_X_R, y, n_repeats=10, random_state=0).importances_mean)
R_gradient_boosting_importances = np.argsort(permutation_importance(gradient_boosting, new_X_R, y, n_repeats=10, random_state=0).importances_mean)
R_svm_importances = np.argsort(permutation_importance(svm, new_X_R, y, n_repeats=10, random_state=0).importances_mean)

In [ ]:
R_lead = np.array([
	R_random_forest_importances,
	R_logistic_regression_importances,
	R_gradient_boosting_importances,
	R_svm_importances
])
R_lead

In [ ]:
R_robustness_score = leaderboards_to_importance(R_lead)
R_robustness_score

## Results

In [ ]:
index = int(X.shape[-1] - np.round(original_dataset_size - original_dataset_size * PERCENTAGE_FEATURE_TO_REMOVE))
accuracies_filtered_together_PI = np.array(accuracies_filtered_together_PI)
accuracies_filtered_together_LOCO = np.array(accuracies_filtered_together_LOCO)


result = np.array([
	[
		PI_robustness_score,
		LOCO_robustness_score,
		PCA_robustness_score,
		KPCA_robustness_score,
		SPCA_robustness_score,
		TSVD_robustness_score,
		FA_robustness_score,
		R_robustness_score,
	],[
		accuracies_filtered_together_PI[index].mean(),
		accuracies_filtered_together_LOCO[index].mean(),
		accuracies_filtered_PCA_BS[index].mean(),
		accuracies_filtered_KPCA[index].mean(),
		accuracies_filtered_SparsePCA[index].mean(),
		accuracies_filtered_trunc_SVD[index].mean(),
		accuracies_filtered_factor_analysis[index].mean(),
		accuracies_with_relief_selection[index].mean(),
	]
])
result

## Conclusions

At the first preliminary meeting the following have been concluded:
- makes no sense to test this method on "too simple" datasets, aka those with intrinsic high correlation inside them
- the following are the best techniques:
	- BS-LOCO
	- BS-FA / BS-Relief
- apply those best techniques to all the models available
- check if the "reason of lying" affects "how do you lie" (the )
- using an autoencoder with permutation importance as "FS with model independent without classifiers", and testing on non-linear classifiers

## NN

In [ ]:
import tensorflow as tf
import keras.api._v2.keras as K

In [ ]:
X_one_hot = K.utils.to_categorical(X)

In [ ]:
K.models.Sequential([
	K.layers.Flatten(input_shape=X_one_hot.shape[1:]),
	K.layers.Dense(64, activation=tf.nn.leaky_relu)
])